In [102]:
import pandas as pd
import json
import re
import json
from bs4 import BeautifulSoup
import bs4
import requests
import os
from tqdm import tqdm
import pickle
from law_query_private import LawQuery,Tree
from slugify import slugify
import gzip
import lawquery

In [103]:
def get_text(law_id):
    # get metadata
    url='https://thuvienphapluat.vn/van-ban/a/a-{}.aspx'.format(law_id)
    metadata_url='https://thuvienphapluat.vn/AjaxLoadData/LoadLuocDo.aspx?LawID={}&IstraiNghiem=True'.format(law_id)
    req=requests.get(metadata_url,headers={'referer': url})
    if req.status_code != 200:
        raise Exception('Error when get metadata')
    soup = BeautifulSoup(req.text, 'html.parser')
    metadata = soup.select_one('#viewingDocument')
    metadata = {
        'ten_van_ban': metadata.select_one('#viewingDocument > div:nth-child(1)').getText(strip=True), 
        'so_hieu_van_ban': metadata.select_one('#viewingDocument > div:nth-child(2) > div.ds.fl').getText(strip=True),
        'loai_van_ban': metadata.select_one('#viewingDocument > div:nth-child(3) > div.ds.fl').getText(strip=True),
        'linhvuc': metadata.select_one('#viewingDocument > div:nth-child(4) > div.ds.fl').getText(strip=True),
        'noi_ban_hanh': metadata.select_one('#viewingDocument > div:nth-child(5) > div.ds.fl').getText(strip=True),
        'nguoi_ky': metadata.select_one('#viewingDocument > div:nth-child(6) > div.ds.fl').getText(strip=True),
        'ngay_ban_hanh': metadata.select_one('#viewingDocument > div:nth-child(7) > div.ds.fl').getText(strip=True),
        'ngay_hieu_luc': metadata.select_one('#viewingDocument > div:nth-child(8) > div.ds.fl').getText(strip=True),
        'ngay_cong_bao': metadata.select_one('#viewingDocument > div:nth-child(9) > div.ds.fl').getText(strip=True),
        'so_cong_bao': metadata.select_one('#viewingDocument > div:nth-child(10) > div.ds.fl').getText(strip=True),
        'tinh_trang': metadata.select_one('#viewingDocument > div:nth-child(11) > div.ds.fl').getText(strip=True),
    }
    req=requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')
    html = soup.find('div', {'class':'content1'})
    for element in html(text=lambda text: isinstance(text, bs4.Comment)):
        element.extract()
    for a in html.find_all('a', href=lambda x: x and x[0] == '#'):
        a.extract()
    for s in html.find_all(['script','style']):
        s.extract()
    ps = html.find_all(['p','h1','h2','h3','h4','h5','h6'])
    content = ''
    for p in ps:
        text = p.text
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = ' '.join(chunk for chunk in chunks if chunk)
        content += text + '\n'
    # delete - more than 3
    content = re.sub(r'-{3,}', '', content)
    content = re.sub(r'\*{3,}', '', content)
    # remove line with empty content
    content = re.sub(r'^\s+$', '', content, flags=re.MULTILINE)
    # name = soup.find('h1').text.strip()
    matches = re.finditer(r'^(chương [\d\w]+.*|phần thứ [\d\w]+.*)$', content, re.MULTILINE | re.IGNORECASE)
    for i,match in enumerate(matches):
        end = match.end()
        while end < len(content) and content[end] != '\n':
            end += 1
        if end < len(content) and content[end] == '\n':
            content = content[:end] + ':' + content[end+1:]
    return {
        'metadata':metadata,
        'text':content,
        'html':html.prettify()
    }

In [104]:
LAW_IDS=['259700', '454887', '299644', '279974', '357505', '268405', '215628', '333670', '341271', '280020', '489758', '81884', '238506', '411816', '393789', '297783', '443310', '281961', '187083', '142187']

In [116]:
# items = []
# for law_id in LAW_IDS:
#     data = get_text(law_id)
#     metadata = data['metadata']
#     text=data['text']
#     html=data['html']
#     print(metadata['ten_van_ban'])
#     name_slug = slugify(metadata['ten_van_ban'])[:50]
#     items.append({'metadata':metadata,'path':name_slug, 'tvpl_id':law_id})
#     folder_path = os.path.join('documents', name_slug)
#     os.makedirs(folder_path, exist_ok=True)
#     with open(os.path.join(folder_path, 'content.txt'), 'w', encoding='utf-8') as f:
#         f.write(text)
#     with open(os.path.join(folder_path, 'content.html'), 'w', encoding='utf-8') as f:
#         f.write(html)

# df = pd.DataFrame(items)
# # expose metadata
# df = pd.concat([df, df['metadata'].apply(pd.Series)], axis=1)
# df['metadata'] = df['metadata'].apply(lambda x: json.dumps(x, ensure_ascii=False))
# df.to_json('documents/data.jsonl', lines=True,orient='records',force_ascii=False)

Luật Bảo hiểm xã hội 2014
Văn bản hợp nhất 2089/VBHN-BHXH năm 2020 hợp nhất Quyết định về Quy trình thu bảo hiểm xã hội, bảo hiểm y tế, bảo hiểm thất nghiệp, bảo hiểm tai nạn lao động, bệnh nghề nghiệp; quản lý sổ bảo hiểm xã hội, thẻ bảo hiểm y tế do Bảo hiểm xã hội Việt Nam ban hành
Thông tư 59/2015/TT-BLĐTBXH quy định chi tiết và hướng dẫn thi hành một số điều của Luật bảo hiểm xã hội về bảo hiểm xã hội bắt buộc do Bộ trưởng Bộ Lao động - Thương binh và Xã hội ban hành
Nghị định 115/2015/NĐ-CP hướng dẫn Luật bảo hiểm xã hội về bảo hiểm xã hội bắt buộc
Nghị định 146/2018/NĐ-CP hướng dẫn Luật bảo hiểm y tế
Nghị định 28/2015/NĐ-CP hướng dẫn Luật Việc làm về bảo hiểm thất nghiệp
Luật việc làm 2013
Bộ luật Lao động 2019
Thông tư 56/2017/TT-BYT về hướng dẫn Luật bảo hiểm xã hội và Luật an toàn vệ sinh lao động thuộc lĩnh vực y tế do Bộ trưởng Bộ Y tế ban hành
Nghị định 134/2015/NĐ-CP hướng dẫn Luật Bảo hiểm xã hội về bảo hiểm xã hội tự nguyện
Quyết định 28/2021/QĐ-TTg quy định về thực hiệ

C:\Users\ngoph\AppData\Local\Temp\ipykernel_1768\121243509.py:19: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  df = pd.concat([df, df['metadata'].apply(pd.Series)], axis=1)


In [117]:
def make_content_for_tree(text):
    re_footer = r'^(Bộ luật này đã được Quốc hội .*|Luật này được Quốc hội.*|Bộ luật này được Quốc hội .*|Luật này đã được Quốc hội .*|Nơi nhận.*)'
    matches = re.split(re_footer, text, flags=re.MULTILINE)
    text = matches[0]
    return text

In [154]:
df = pd.read_json('documents/data.jsonl',lines=True)
law_iter = iter(df.itertuples())

In [156]:
row = next(law_iter)
row

Pandas(Index=0, metadata='{"ten_van_ban": "Luật Bảo hiểm xã hội 2014", "so_hieu_van_ban": "58/2014/QH13", "loai_van_ban": "Luật", "linhvuc": "Bảo hiểm, Lao động - Tiền lương", "noi_ban_hanh": "Quốc hội", "nguoi_ky": "Nguyễn Sinh Hùng", "ngay_ban_hanh": "20/11/2014", "ngay_hieu_luc": "01/01/2016", "ngay_cong_bao": "29/12/2014", "so_cong_bao": "Từ số 1163 đến số 1164", "tinh_trang": "Còn hiệu lực"}', path='luat-bao-hiem-xa-hoi-2014', tvpl_id=259700, ten_van_ban='Luật Bảo hiểm xã hội 2014', so_hieu_van_ban='58/2014/QH13', loai_van_ban='Luật', linhvuc='Bảo hiểm, Lao động - Tiền lương', noi_ban_hanh='Quốc hội', nguoi_ky='Nguyễn Sinh Hùng', ngay_ban_hanh='20/11/2014', ngay_hieu_luc='01/01/2016', ngay_cong_bao='29/12/2014', so_cong_bao='Từ số 1163 đến số 1164', tinh_trang='Còn hiệu lực')

In [157]:
CREATE_CONTENT_TREE=True

print('https://thuvienphapluat.vn/van-ban/a/a-{}.aspx'.format(row.tvpl_id))
print(os.path.join(os.getcwd(),'documents', row.path,'debug_tree.txt'))
folder_path = os.path.join('documents', row.path)
text = open(os.path.join(folder_path, 'content.txt'), encoding='utf-8').read()
raw_text = text
text = make_content_for_tree(text)
if CREATE_CONTENT_TREE:
    with open(os.path.join(folder_path, 'content_tree.txt'), 'w', encoding='utf-8') as f:
        f.write(text)
text = open(os.path.join(folder_path, 'content_tree.txt'), encoding='utf-8').read()
tree = Tree(metadata= json.loads(row.metadata),content=text,raw_text=raw_text)
with open(os.path.join(folder_path, 'debug_tree.txt'), 'w', encoding='utf-8') as f:
    f.write(str(tree))
with open(os.path.join(folder_path, 'tree.pkl'), 'wb') as f:
    pickle.dump(tree, f)
with open(os.path.join(folder_path, 'tree.json'), 'w', encoding='utf-8') as f:
    f.write(json.dumps(tree.export(), indent=4, ensure_ascii=False))
with open(os.path.join(folder_path, 'tree.json.gz'), 'wb') as f:
    f.write(gzip.compress(json.dumps(tree.export(), ensure_ascii=False).encode('utf-8')))
engine = lawquery.Engine(os.path.join(folder_path,'tree.json.gz'))
print(len(engine.query(node_type='điều')))
engine.query(node_type='điều')

https://thuvienphapluat.vn/van-ban/a/a-259700.aspx
c:\Users\ngoph\Desktop\luanvantotnghiep\code\documents\luat-bao-hiem-xa-hoi-2014\debug_tree.txt
125


[Điều 1. Phạm vi điều chỉnh,
 Điều 2. Đối tượng áp dụng,
 Điều 3. Giải thích từ ngữ,
 Điều 4. Các chế độ bảo hiểm xã hội,
 Điều 5. Nguyên tắc bảo hiểm xã hội,
 Điều 6. Chính sách của Nhà nước đối với bảo hiểm xã hội,
 Điều 7. Nội dung quản lý nhà nước về bảo hiểm xã hội,
 Điều 8. Cơ quan quản lý nhà nước về bảo hiểm xã hội,
 Điều 9. Hiện đại hóa quản lý bảo hiểm xã hội,
 Điều 10. Trách nhiệm của Bộ trưởng Bộ Lao động - Thương binh và Xã hội về bảo hiểm xã hội,
 Điều 11. Trách nhiệm của Bộ trưởng Bộ Tài chính về bảo hiểm xã hội,
 Điều 12. Trách nhiệm của Ủy ban nhân dân các cấp về bảo hiểm xã hội,
 Điều 13. Thanh tra bảo hiểm xã hội,
 Điều 14. Quyền và trách nhiệm của tổ chức công đoàn, Mặt trận Tổ quốc Việt Nam và các tổ chức thành viên của Mặt trận,
 Điều 15. Quyền và trách nhiệm của tổ chức đại diện người sử dụng lao động,
 Điều 16. Chế độ báo cáo, kiểm toán,
 Điều 17. Các hành vi bị nghiêm cấm,
 Điều 18. Quyền của người lao động,
 Điều 19. Trách nhiệm của người lao động,
 Điều 20. Q